In [1]:
from mlops.project_libraries import *
from mlops.util_funcs import *

In [2]:
# fill in AWS profile
os.environ["AWS_PROFILE"] = "demiga-g"

# Setting tracking uri (unique resource identifier)
TRACKING_SERVER_HOST =  '127.0.0.1'  # '16.171.136.194'
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")
# mlflow.set_experiment("model-vectorizer-as-one")

In [3]:
# load data
new_data = pd.read_csv('../data/val_df1.csv')

# scrub the data
df = scrub_data(new_data).to_dict(orient='records')

In [4]:
# get the model
BASE_LOCATION = 'mlflow-artifacts:' # 's3://midega-mlflow-artifacts'
EXPERIMENT_ID = 6
RUN_ID = '45a3990ff1e140afbe48334a8422bec7'
logged_model = f'{BASE_LOCATION}/{EXPERIMENT_ID}/{RUN_ID}/artifacts/model'
model = mlflow.pyfunc.load_model(logged_model)

/home/midega-g/anaconda3/envs/ifood_mlops/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# predictions
y_pred = model.predict(df)

In [6]:
# store data in a dataframe
df_result = pd.DataFrame()
df_result['ID'] = new_data['ID']
df_result['PredictedResponse'] = y_pred
df_result['ModelVersion'] = RUN_ID

In [7]:
# compare results with actual responses
df_result['ActualResponse'] = new_data['Response']
df_result['PredictionStatus'] = ((df_result['PredictedResponse'] == df_result['ActualResponse'])
                                 .map({True: "Correct", False: "Incorrect"}))
overall_accuracy = df_result['PredictionStatus'].value_counts(normalize=True)
positive_response_accuracy = len(df_result[(df_result['PredictedResponse']==1) & 
                                       (df_result['ActualResponse']== 1)])

In [8]:
overall_accuracy

Correct      0.693069
Incorrect    0.306931
Name: PredictionStatus, dtype: float64

In [9]:
positive_response_accuracy

42

In [10]:
df_result['ActualResponse'].value_counts()

0    563
1     43
Name: ActualResponse, dtype: int64